# Riiid! Answer Correctness Prediction

The notebook can be treated as a infrence notebook i have commented the training part here if you can find the model here - https://www.kaggle.com/ash1706/model-cat-riid.

The aim is to use LGBM and catboost and then use thier wieghted average to make the predictions.

I just started with the competetion will work on this as i get more ideas.

## please leave a upvote !!!!!!!  😁🙏

I am using the features generated by - https://www.kaggle.com/ldevyataykina/riiid-exploratory-data-analysis-baseline <br>
and thanks to this kernel for helping in loading the huge dataset it was a pain - https://www.kaggle.com/rohanrao/tutorial-on-reading-large-datasets/

In [ ]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, learning_curve
from sklearn.utils import shuffle
import lightgbm as lgb
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier
import xgboost as xgb


import eli5

import riiideducation
import joblib
import time

%matplotlib inline
# for heatmap and other plots
colorMap1 = sns.color_palette("RdBu_r")
# for countplot and others plots
colorMap2 = 'Blues_r'

import os
for dirname, _, filenames in os.walk('/kaggle/input/riiid-test-answer-prediction'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#### PATHS

In [ ]:
train_path = "../input/riiid-train-data-multiple-formats/riiid_train.pkl.gzip"
questions_path = "../input/riiid-test-answer-prediction/questions.csv"
lectures_path = "../input/riiid-test-answer-prediction/lectures.csv"

test = "../input/riiid-test-answer-prediction/example_test.csv"

### TRAIN

>**row_id**: (int64) ID code for the row.

>**timestamp**: (int64) the time in milliseconds between this user interaction and the first event completion from that user.

>**user_id**: (int32) ID code for the user.

>**content_id**: (int16) ID code for the user interaction

>**content_type_id**: (int8) 0 if the event was a question being posed to the user, 1 if the event was the user watching a lecture.

>**task_container_id**: (int16) Id code for the batch of questions or lectures. For example, a user might see three questions in a row before seeing the explanations for any of them. Those three would all share a task_container_id.

>**user_answer**: (int8) the user's answer to the question, if any. Read -1 as null, for lectures.

>**answered_correctly**: (int8) if the user responded correctly. Read -1 as null, for lectures.

>**prior_question_elapsed_time**: (float32) The average time in milliseconds it took a user to answer each question in the previous question bundle, ignoring any lectures in between. Is null for a user's first question bundle or lecture. Note that the time is the average time a user took to solve each question in the previous bundle.

>**prior_question_had_explanation**: (bool) Whether or not the user saw an explanation and the correct response(s) after answering the previous question bundle, ignoring any lectures in between. The value is shared across a single question bundle, and is null for a user's first question bundle or lecture. Typically the first several questions a user sees were part of an onboarding diagnostic test where they did not get any feedback.


In [ ]:

%%time

df = pd.read_pickle(train_path)

In [ ]:
print(f"Train shape: {df.shape}")

In [ ]:
df.head(10)

The difference between **timestamp** is not equal to **prior_question_elapsed_time**. Therefore we need to create **timestamp** difference and check it importance

In [ ]:
df['timestamp_diff'] = df['timestamp'].diff()
df['timestamp_diff'].fillna(0, inplace=True)
df.loc[df['timestamp_diff'] < 0, 'timestamp_diff'] = np.median(df['timestamp_diff'])

In [ ]:
df.memory_usage()

Droping **row_id** and **timestamp** 

In [ ]:
df.drop(['row_id', 'timestamp'], axis=1, inplace=True)

In [ ]:
df.describe()

In [ ]:
print(f'Number of unique users: {len(np.unique(df.user_id))}')

Let's look at the missing data

In [ ]:
print(df.isnull().sum() / len(df))

In [ ]:
freq_answered_tasks = df['task_container_id'].value_counts().reset_index()
freq_answered_tasks.columns = [
    'task_container_id', 
    'freq'
]

df['freq_task_id'] = ''
df.loc[df['task_container_id'].isin(freq_answered_tasks[freq_answered_tasks['freq'] < 10000]['task_container_id'].values), 'freq_task_id'] = 'very rare answered'
df.loc[df['task_container_id'].isin(freq_answered_tasks[freq_answered_tasks['freq'] >= 10000]['task_container_id'].values), 'freq_task_id'] = 'rare answered'
df.loc[df['task_container_id'].isin(freq_answered_tasks[freq_answered_tasks['freq'] >= 50000]['task_container_id'].values), 'freq_task_id'] = 'normal answered'
df.loc[df['task_container_id'].isin(freq_answered_tasks[freq_answered_tasks['freq'] >= 200000]['task_container_id'].values), 'freq_task_id'] = 'often answered'
df.loc[df['task_container_id'].isin(freq_answered_tasks[freq_answered_tasks['freq'] >= 400000]['task_container_id'].values), 'freq_task_id'] = 'very often answered'

In [ ]:
df.sample(5)

Change the -1 values to 0 for lectures in **user_answer** and **answered_correctly** as we saw in the columns' description

In [ ]:
df.loc[(df['content_type_id'] == 1) & (df['user_answer'] == -1), 'user_answer'] = 0
df.loc[(df['content_type_id'] == 1) & (df['answered_correctly'] == -1), 'answered_correctly'] = 0

Let's look where is the most part of the incorrect answers

In [ ]:
df.groupby(['content_type_id', 'answered_correctly']).agg({'answered_correctly': 'count'})

### QUESTIONS.CSV

>**question_id**: foreign key for the train/test content_id column, when the content type is question (0).

>**bundle_id**: code for which questions are served together.

>**correct_answer**: the answer to the question. Can be compared with the train user_answer column to check if the user was right.

>**part**: top level category code for the question.

>**tags**: one or more detailed tag codes for the question. The meaning of the tags will not be provided, but these codes are sufficient for clustering the questions together.

In [ ]:
questions = pd.read_csv(questions_path)
questions.head(10)

In [ ]:
questions.describe().style.background_gradient(cmap='Blues')

In [ ]:
print(questions.isnull().sum() / len(questions))

Let's check the parts distribution

### LECTURES.CSV

Metadata for the lectures watched by users as they progress in their education.

>**lecture_id**: foreign key for the train/test content_id column, when the content type is lecture (1).

>**part**: top level category code for the lecture.

>**tag**: one tag codes for the lecture. The meaning of the tags will not be provided, but these codes are sufficient for clustering the lectures together.

>**type_of**: brief description of the core purpose of the lecture

In [ ]:
lectures = pd.read_csv(lectures_path)
lectures.head(10)

In [ ]:
lectures['type_of'].value_counts()

### EXAMPLE-TEST.CSV

In [ ]:
test_example = pd.read_csv(test)

In [ ]:
test_example.head(10)

<a id="2"></a>

<div style="background:write; border:0; color:black; width: 100%; height: 50px">
    <div style="vertical-align: middle; text-align:center"><h1>FEATURE ENGINEERING</h1></div>
</div>

In [ ]:
n = int(df.shape[0] * 0.1)
train = df.sample(n=n, random_state=42)

In [ ]:
del questions
del lectures


In [ ]:
user_characteristics = df.groupby('user_id').agg({'answered_correctly':
                                                  ['mean', 'median', 'std', 'skew', 'count']})
user_characteristics.columns = [
    'mean_user_acc',
    'median_user_acc',
    'std_user_acc',
    'skew_user_acc',
    'number_of_answered_q'
]

We saw earlier some dependencies between **answered_correctly** and the frequency of **task_container_id**. Therefore I want to add some features for the **task_container_id**

In [ ]:
task_container_characteristics = df.groupby('task_container_id').agg({'answered_correctly':
                                                                      ['mean', 'median', 'std', 'skew', 'count']})
task_container_characteristics.columns = [
    'mean_task_acc',
    'median_task_acc',
    'std_task_acc',
    'skew_task_acc',
    'number_of_asked_task_containers'
]

In [ ]:
task_container_characteristics.head(5)

In [ ]:
content_characteristics = df.groupby('content_id').agg({'answered_correctly':
                                                        ['mean', 'median', 'std', 'skew', 'count']})
content_characteristics.columns = [
    'mean_acc',
    'median_acc',
    'std_acc',
    'skew_acc',
    'number_of_asked_q'
]

In [ ]:
# content_characteristics.head(5)

In [ ]:
df = train.copy()
del train

Merge all of our data

In [ ]:
df = df.merge(user_characteristics, how='left', on='user_id')
df = df.merge(task_container_characteristics, how='left', on='task_container_id')
df = df.merge(content_characteristics, how='left', on='content_id')

In [ ]:
features = [
    'prior_question_elapsed_time', 
    'prior_question_had_explanation',
    'timestamp_diff',
    'mean_user_acc',
    'median_user_acc',
    'std_user_acc',
    'skew_user_acc',
    'number_of_answered_q',
    'mean_task_acc',
    'median_task_acc',
    'std_task_acc',
    'skew_task_acc',
    'number_of_asked_task_containers',
    'mean_acc',
    'median_acc',
    'std_acc',
    'skew_acc',
    'number_of_asked_q'
]

target = 'answered_correctly'

Drop features that we are not going to use in our model

In [ ]:
col_to_drop = set(df.columns.values.tolist()).difference(features + [target])
for col in col_to_drop:
    del df[col]

In [ ]:
df['prior_question_had_explanation'] = df['prior_question_had_explanation'].fillna(value=False).astype(bool)
df = df.fillna(value=0.5)

In [ ]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0.5)

In [ ]:
df.head(5)

<a id="3"></a>

<div style="background:write; border:0; color:black; width: 100%; height: 50px">
    <div style="vertical-align: middle; text-align:center"><h1>MODELING</h1></div>
</div>

In [ ]:
train_df, test_df, y_train, y_test = train_test_split(df[features], df[target], random_state=777, test_size=0.3)

The models are already trained and present in - https://www.kaggle.com/ash1706/model-cat-riid you can add it and play around . the models are basleline though nothing fancy.
The params are given in below cells in case you want to train it again

In [ ]:
# hyper parametre optimization
# clf = LGBMClassifier(random_state=777)

# params = {
#     'n_estimators': [50, 150, 300],
#     'max_depth': [3, 5, 10],
#     'num_leaves': [5, 15, 30],
#     'min_data_in_leaf': [5, 50, 100],
#     'feature_fraction': [0.1, 0.5, 1.],
#     'lambda': [0., 0.5, 1.],
# }

# cv = RandomizedSearchCV(clf, param_distributions=params, cv=5, n_iter=50, verbose=2)
# cv.fit(df[features], df[target])

# print(cv.best_params_)
# print(cv.best_score_)

It was the best params. Therefore I will use them

In [ ]:
params = {
    'num_leaves': 10, 
    'n_estimators': 100, 
    'min_data_in_leaf': 10, 
    'max_depth': 5, 
    'lambda': 0.0, 
    'feature_fraction': 1.0 , 
    'device': 'gpu',
    'gpu_platform_id': 0,
    'gpu_device_id': 0
}

In [ ]:

params_cat = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'task_type': 'GPU' ,
    'grow_policy': 'Lossguide',
    'iterations': 2500,
    'learning_rate': 4e-2,
    'random_seed': 0,
    'l2_leaf_reg': 1e-1,
    'depth': 15,
    'max_leaves': 10,
    'border_count': 128,
    'verbose': 50,
}

In [ ]:
# train_df.prior_question_had_explanation.value_counts()

In [ ]:
# models lgbm and catboost uncomment to train
model_lbgm = LGBMClassifier(**params)
# model_lbgm.fit(train_df, y_train)
model =  CatBoostClassifier(**params_cat)
# model_lbgm.fit(train_df, y_train)

In [ ]:
# model.save_model("model.cbm") # save model

In [ ]:
# import joblib
# joblib.dump(model_lbgm, 'lgb.pkl')



In [ ]:
lgb_model = joblib.load('../input/model-cat-riid/lgb.pkl')# load model

In [ ]:
model.load_model('../input/model-cat-riid/model.cbm') #load catboost



# CV

In [ ]:
print('LGB ROC-AUC score: ', roc_auc_score(y_test.values, model.predict_proba(test_df)[:, 1]))

In [ ]:
print('LGB ROC-AUC score: ', roc_auc_score(y_test.values, lgb_model.predict_proba(test_df)[:, 1]))

In [ ]:
print('LGB ROC-AUC score: ', roc_auc_score(y_test.values, model.predict_proba(test_df)[:, 1] * 0.8 + lgb_model.predict_proba(test_df)[:, 1] * 0.2))
#  model.predict_proba(test_df)[:, 1] * 0.8 + lgb_model.predict_proba(test_df)[:, 1] * 0.2

In [ ]:
eli5.show_weights(model, top=20)

to many useless features work on this

# weight optimization

In [ ]:
weights = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
roc_auc = []
for i in weights:
    roc_auc.append(roc_auc_score(y_test.values, model.predict_proba(test_df)[:, 1] * i + lgb_model.predict_proba(test_df)[:, 1] * (1-i)))
    
    

In [ ]:
sns.lineplot(x = weights, y = roc_auc  )

0.9 seems to be the best 

<a id="4"></a>

<div style="background:write; border:0; color:black; width: 100%; height: 50px">
    <div style="vertical-align: middle; text-align:center"><h1>SUBMISSION PREPARATION</h1></div>
</div>

In [ ]:
env = riiideducation.make_env()


In [ ]:
iter_test = env.iter_test()

In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    # new features
    test_df['timestamp_diff'] = test_df['timestamp'].diff()
    test_df['timestamp_diff'].fillna(0, inplace=True)
    test_df.loc[test_df['timestamp_diff'] < 0, 'timestamp_diff'] = np.median(test_df['timestamp_diff'])
    
    # merge
    test_df = test_df.merge(user_characteristics, on = "user_id", how = "left")
    test_df = test_df.merge(task_container_characteristics, on = "task_container_id", how = "left")
    test_df = test_df.merge(content_characteristics, on = "content_id", how = "left")
    
    # type transformation
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(value=False).astype(bool)
    test_df.fillna(value = 0.5, inplace = True)
    test_df = test_df.replace([np.inf, -np.inf], np.nan)
    test_df = test_df.fillna(0.5)
    
    # preds
    test_df['answered_correctly'] = model.predict_proba(test_df[features])[:, 1]* 0.9 + lgb_model.predict_proba(test_df[features])[:, 1] * 0.1
    cols_to_submission = ['row_id', 'answered_correctly', 'group_num']
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])

Thanks for the read !!!!<br>
please do leave an upvote!!!!!

# to do
1. Throw SAKT in the mix seems like the way to go
2. Work on feature eginnering